In [36]:
# Import packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
import pickle as pkl

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_curve, auc

In [12]:
path = "C:/Users/gsamp/OneDrive/Documents/AI-3/2n Semestre/Projecte de Síntesi 2/GraphAnomaly/dades_guillem/"
df_classes = pd.read_csv(path + "elliptic_txs_classes.csv") # Nodes' labels
df_edges = pd.read_csv(path + "elliptic_txs_edgelist.csv") # Edges
df_features = pd.read_csv(path + "elliptic_txs_features.csv", header=None) # Nodes' features
df_features = df_features.rename(columns={0:"txId"})

In [2]:
# Specify the path to your pickle file
pickle_file_path = 'C:\\Users\\gsamp\\OneDrive\\Documents\\AI-3\\2n Semestre\\Projecte de Síntesi 2\\GraphAnomaly\\elipticData_graph.pkl'

# Open the pickle file in binary mode
with open(pickle_file_path, 'rb') as f:
    # Load the data from the pickle file
    G = pkl.load(f)


In [42]:
df_merged = pd.merge(left=df_features, right=df_classes, on="txId", how="left")
df_merged = df_merged.set_index('txId')

APPLY PCA

In [85]:
# Standardize the features (important for PCA)
scaler = StandardScaler()

df_pca = df_merged.drop(columns=['class'])
scaled_data = scaler.fit_transform(df_pca)

# Apply PCA
pca = PCA(n_components=100)  # You can choose the number of components you want to keep
principal_components = pca.fit_transform(scaled_data)

# Create a DataFrame for the principal components
columns = [f"PC{i+1}" for i in range(principal_components.shape[1])]
principal_df = pd.DataFrame(data=principal_components, columns=columns)


explained_variance_ratio = pca.explained_variance_ratio_
cumulative_variance_ratio = explained_variance_ratio.sum()

print(f"Explained variance ratio: {cumulative_variance_ratio}")
print(f"Data reduction, from shape {df_pca.shape} to {principal_df.shape}")


Explained variance ratio: 0.9985460497241709
Data reduction, from shape (203769, 166) to (203769, 100)


Logistic regression

In [86]:
#afegir dues columnes per poder aplicar logistic regression
principal_df['txId'] = df_pca.index
principal_df['class'] = list(df_merged['class'])

X = principal_df.loc[principal_df['class'].isin(['1', '2'])].drop(columns=['txId', 'class'])
y = principal_df.loc[principal_df['class'].isin(['1', '2'])]['class']
print(set(list(principal_df.loc[principal_df['class'].isin(['1', '2'])]['class'])))
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

# Train the Logistic Regression model
model = LogisticRegression()
model.fit(X_train, y_train)

# Evaluate the model
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: {:.2f}%".format(accuracy * 100))

{'2', '1'}
Accuracy: 96.02%


c:\Users\gsamp\OneDrive\Documents\AI-3\2n Semestre\Projecte de Síntesi 2\GraphAnomaly\sintesis_venv\lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [82]:
print(f"Number of class 1 {list(y).count('1')}")
print(f"Number of class 2 {list(y).count('2')}")            

Number of class 1 4545
Number of class 2 42019
